### Best score is coming at missing -999, non-normalised, > 96% dropping features : 66% something

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD, PCA, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline, make_union
from sklearn.manifold import TSNE

In [ ]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 10000)

In [ ]:
with open('train_iI26erW/train.csv', 'r') as fp, open('test_plBmD8c.csv') as fg:
    train = pd.read_csv(fp)
    test = pd.read_csv(fg)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
## count no. of NA per row

train['na_count'] = train.isnull().sum(axis=1)
test['na_count'] = test.isnull().sum(axis=1)

In [ ]:
## net balance after spending in mon 1
addCols = [x for x in train.columns if 'CARD_' in x]
train['add_spend_mon1'] = train[addCols].sum(axis = 1) 
test['add_spend_mon1'] = test[addCols].sum(axis = 1)

train['add_spend_mon1'] = train['add_spend_mon1'].map(lambda x: x.count('Y'))
test['add_spend_mon1'] = test['add_spend_mon1'].map(lambda x: x.count('Y'))

In [ ]:
train['CLOSED_DATE'] = train['CLOSED_DATE'].astype(str)
test['CLOSED_DATE'] = test['CLOSED_DATE'].astype(str)

In [ ]:
def getDateFormat(val):
    if not val == 'nan':
        try:
            a = datetime.strptime(val, '%d%b%Y')
            b = a.strftime('%Y-%m-%d')
            return b
        except TypeError:
            return np.nan
    else:
        return np.nan

In [ ]:
from datetime import datetime

train['closed_date_format'] = train['CLOSED_DATE'].map(getDateFormat)
test['closed_date_format'] = test['CLOSED_DATE'].map(getDateFormat)

train['maturity_gl_format'] = train['MATURITY_GL'].map(getDateFormat)
test['maturity_gl_format'] = test['MATURITY_GL'].map(getDateFormat)

train['maturity_lap_format'] = train['MATURITY_LAP'].map(getDateFormat)
test['maturity_lap_format'] = test['MATURITY_LAP'].map(getDateFormat)

train['maturity_las_format'] = train['MATURITY_LAS'].map(getDateFormat)
test['maturity_las_format'] = test['MATURITY_LAS'].map(getDateFormat)

In [ ]:
## drop date formats
train.drop(['CLOSED_DATE','MATURITY_GL','MATURITY_LAP','MATURITY_LAS'], axis=1, inplace=True)
test.drop(['CLOSED_DATE','MATURITY_GL','MATURITY_LAP','MATURITY_LAS'], axis=1, inplace=True)

In [ ]:
## date features

train['closed_date_format'] = pd.to_datetime(train['closed_date_format'])
test['closed_date_format'] = pd.to_datetime(test['closed_date_format'])

train['maturity_gl_format'] = pd.to_datetime(train['maturity_gl_format'])
test['maturity_gl_format'] = pd.to_datetime(test['maturity_gl_format'])

train['maturity_lap_format'] = pd.to_datetime(train['maturity_lap_format'])
test['maturity_lap_format'] = pd.to_datetime(test['maturity_lap_format'])

train['maturity_las_format'] = pd.to_datetime(train['maturity_las_format'])
test['maturity_las_format'] = pd.to_datetime(test['maturity_las_format'])

In [ ]:
train['closed_date_weekday'] = train['closed_date_format'].dt.weekday
test['closed_date_weekday'] = test['closed_date_format'].dt.weekday

train['closed_date_year'] = train['closed_date_format'].dt.year
test['closed_date_year'] = test['closed_date_format'].dt.year

train['closed_date_month'] = train['closed_date_format'].dt.month
test['closed_date_month'] = test['closed_date_format'].dt.month

train['closed_date_day'] = train['closed_date_format'].dt.day
test['closed_date_day'] = test['closed_date_format'].dt.day

#########################################################

train['maturity_gl_weekday'] = train['maturity_gl_format'].dt.weekday
test['maturity_gl_weekday'] = test['maturity_gl_format'].dt.weekday

train['maturity_gl_year'] = train['maturity_gl_format'].dt.year
test['maturity_gl_year'] = test['maturity_gl_format'].dt.year

train['maturity_gl_month'] = train['maturity_gl_format'].dt.month
test['maturity_gl_month'] = test['maturity_gl_format'].dt.month

train['maturity_gl_day'] = train['maturity_gl_format'].dt.day
test['maturity_gl_day'] = test['maturity_gl_format'].dt.day

################################################################

train['maturity_lap_weekday'] = train['maturity_lap_format'].dt.weekday
test['maturity_lap_weekday'] = test['maturity_lap_format'].dt.weekday

train['maturity_lap_year'] = train['maturity_lap_format'].dt.year
test['maturity_lap_year'] = test['maturity_lap_format'].dt.year

train['maturity_lap_month'] = train['maturity_lap_format'].dt.month
test['maturity_lap_month'] = test['maturity_lap_format'].dt.month

train['maturity_lap_day'] = train['maturity_lap_format'].dt.day
test['maturity_lap_day'] = test['maturity_lap_format'].dt.day

######################################################################

train['maturity_las_weekday'] = train['maturity_las_format'].dt.weekday
test['maturity_las_weekday'] = test['maturity_las_format'].dt.weekday

train['maturity_las_year'] = train['maturity_las_format'].dt.year
test['maturity_las_year'] = test['maturity_las_format'].dt.year

train['maturity_las_month'] = train['maturity_las_format'].dt.month
test['maturity_las_month'] = test['maturity_las_format'].dt.month

train['maturity_las_day'] = train['maturity_las_format'].dt.day
test['maturity_las_day'] = test['maturity_las_format'].dt.day

######################################################################


In [ ]:
formatCols = [x for x in train.columns if  '_format' in x]
train.drop(formatCols, axis=1, inplace=True)
test.drop(formatCols, axis=1, inplace=True)

In [ ]:
train['net_emi_paid'] = train['HL_EMI'] / train['SCRUB_EMI']
test['net_emi_paid'] = test['HL_EMI'] / test['SCRUB_EMI']

train['total_salary_credited'] = train['SAL_MON_01'] + train['SAL_MON_02'] + train['SAL_MON_03']
test['total_salary_credited'] = test['SAL_MON_01'] + test['SAL_MON_02'] + test['SAL_MON_03']

In [ ]:
train['total_balance'] = train['EOP_MON_02'] + train['EOP_MON_03'] + train['EOP_BAL_MON_01']
test['total_balance'] =  test['EOP_MON_02'] + test['EOP_MON_03'] + test['EOP_BAL_MON_01']

In [ ]:
cardCols = [x for x in train.columns if 'CARD_' in x]

for x in cardCols:
    train[x] = train[x].map(lambda x: 1 if x == 'Y' else 0)
    test[x] = test[x].map(lambda x: 1 if x == 'Y' else 0)

In [ ]:
train['spent_sum'] = train[cardCols].sum(axis=1)
test['spent_sum'] = test[cardCols].sum(axis=1)

In [ ]:
dcount = [x for x in train.columns if 'D_COUNT' in x]
ccount = [x for x in train.columns if 'C_COUNT' in x]

train['debit_count'] = train[dcount].sum(axis=1)
train['credit_count'] = train[ccount].sum(axis=1)

test['debit_count'] = test[dcount].sum(axis=1)
test['credit_count'] = test[ccount].sum(axis=1)

In [ ]:
damount = [x for x in train.columns if 'D_AMT' in x]
camount = [x for x in train.columns if 'C_AMT' in x]

train['debit_amount'] = train[damount].sum(axis=1)
train['credit_amount'] = train[camount].sum(axis=1)

test['debit_amount'] = test[damount].sum(axis=1)
test['credit_amount'] = test[camount].sum(axis=1)

In [ ]:
train['DESIGNATION_FINAL'] = train['DESIGNATION_FINAL'].map(lambda x: np.nan if x == 'MISSING' else x)
test['DESIGNATION_FINAL'] = test['DESIGNATION_FINAL'].map(lambda x: np.nan if x == 'MISSING' else x)

In [ ]:
dest_tr = pd.get_dummies(train['DESIGNATION_FINAL'])
det_te = pd.get_dummies(test['DESIGNATION_FINAL'])

In [ ]:
train = pd.concat([train, dest_tr], axis=1)
test = pd.concat([test, det_te], axis=1)

In [ ]:
train.drop('DESIGNATION_FINAL', axis=1, inplace=True)
test.drop('DESIGNATION_FINAL', axis=1, inplace=True)

In [ ]:
ccspend = sorted([x for x in train.columns if 'CC_SPEND' in x])
dcspend = sorted([x for x in train.columns if 'DC_SPEND' in x])

In [ ]:
train['credit_spent'] = train['CC_SPEND_MON_01'] + train['CC_SPEND_MON_02'] + train['CC_SPEND_MON_03'] +train['CC_SPEND_MON_04'] + train['CC_SPEND_MON_05'] + train['CC_SPEND_MON_06'] 
test['credit_spent'] = test['CC_SPEND_MON_01'] + test['CC_SPEND_MON_02'] + test['CC_SPEND_MON_03'] + test['CC_SPEND_MON_04'] + test['CC_SPEND_MON_05'] + test['CC_SPEND_MON_06'] 

train['debit_spent'] = train['DC_SPEND_MON_01'] + train['DC_SPEND_MON_02'] + train['DC_SPEND_MON_03'] +train['DC_SPEND_MON_04'] + train['DC_SPEND_MON_05'] + train['DC_SPEND_MON_06'] 
test['debit_spent'] = test['DC_SPEND_MON_01'] + test['DC_SPEND_MON_02'] + test['DC_SPEND_MON_03'] +test['DC_SPEND_MON_04'] + test['DC_SPEND_MON_05'] + test['DC_SPEND_MON_06'] 

In [ ]:
cats= ['_ATM', '_CARE', '_ENT', '_HBY', '_HMD', '_HTL', '_JER', '_MED', '_RST', '_TRL']

In [ ]:
## total sum by categories
for i in ccspend:
    for j in cats:
        p = []
        if i.endswith(j):
            p.append(i)
        train['cc_sum_'+j] = train[p].sum(axis=1)
        test['cc_sum_'+j] = test[p].sum(axis=1)

In [ ]:
## total sum by debit category
for i in dcspend:
    for j in cats:
        p = []
        if i.endswith(j):
            p.append(i)
        train['dc_sum_'+j] = train[p].sum(axis=1)
        test['dc_sum_'+j] = test[p].sum(axis=1)

In [ ]:
## total credit spent is greater than credit limit

train['is_exceed_credit'] = train.apply(lambda row: 1 if (row['STMT_MON_01'] + row['STMT_MON_02'] + row['STMT_MON_03']) > row['CR_LIMIT'] else 0, axis=1)
test['is_exceed_credit'] = test.apply(lambda row: 1 if (row['STMT_MON_01'] + row['STMT_MON_02'] + row['STMT_MON_03']) > row['CR_LIMIT'] else 0, axis =1)

In [ ]:
## count live, closed loans
liveLoan = [x for x in train.columns if 'SCRUB_LIVE' in x]
closedLoan = [x for x in train.columns if 'SCRUB_CLOSED' in x]

train['live_loan_count'] = train[liveLoan].sum(axis=1)
train['closed_loan_count'] = train[closedLoan].sum(axis=1)

test['live_loan_count'] = test[liveLoan].sum(axis=1)
test['closed_loan_count'] = test[closedLoan].sum(axis=1)

In [ ]:
txn_mon = [x for x in train.columns if 'TXN_MON' in x]
txn_mon = txn_mon[:-1]

In [ ]:
for c in txn_mon:
    p = []
    if 'CC_TXN' in c:
        p.append(c)
    train['total_credit_tran'] = train[p].sum(axis=1)
    test['total_credit_tran'] = test[p].sum(axis=1)
    
for k in txn_mon:
    j = []
    if 'DC_TXN' in k:
        j.append(c)
    train['total_debit_tran'] = train[j].sum(axis=1)
    test['total_debit_tran'] = test[j].sum(axis=1)

In [ ]:
print(train.shape)
print(test.shape)

### one hot encoding

In [ ]:
objCols = train.select_dtypes(include=['object']).columns

In [ ]:
countDict = {}
for x in objCols:
    if train[x].nunique() > 2:
        countDict[x] = train[x].nunique()
        
oneHotCols = list(countDict.keys())

In [ ]:
## check if object columns in both datasets have same unique values

for x in oneHotCols:
    if train[x].nunique() != test[x].nunique():
        set_train = set(train[x].unique())
        set_test = set(test[x].unique())
        
        remove_train = set_train - set_test
        remove_test = set_test - set_train
        
        remove = remove_train.union(remove_test)
        
        def filter_cat(x):
            if x in remove:
                return np.nan
            return x
        
        train[x] = train[x].map(filter_cat)
        test[x] = test[x].map(filter_cat)

In [ ]:
oneHotCols = [x for x in oneHotCols if 'ZIP_CODE' not in x]

In [ ]:
oneHot_train = pd.get_dummies(data=train[oneHotCols])
oneHot_test = pd.get_dummies(data=test[oneHotCols])

In [ ]:
print(oneHot_train.shape)
print(oneHot_test.shape)

In [ ]:
train = pd.concat([train, oneHot_train], axis=1)
test = pd.concat([test, oneHot_test], axis=1)

In [ ]:
train.drop(oneHotCols, axis=1, inplace=True)
test.drop(oneHotCols, axis=1, inplace=True)

In [ ]:
del oneHot_test, oneHot_train

In [ ]:
import gc
gc.collect()

In [ ]:
print(train.shape)
print(test.shape)

### label encoding

In [ ]:
for x in train.columns:
    if x not in test.columns:
        print(x)

In [ ]:
for x in test.columns:
    if x not in train.columns:
        print(x)

In [ ]:
train.drop(['NEFT_CC_CATEGORY_WEDDING','NEFT_DC_CATEGORY_RETURN'], axis=1, inplace=True)
test.drop(['NEFT_CC_CATEGORY_CC_PAYMENT','NEFT_DC_CATEGORY_ADVANCE'], axis=1, inplace=True)

### remove columns with missing values - then label encode

In [ ]:
misCols = train.isnull().sum(axis=0)/train.shape[0]
toremove = []

for k, v in misCols.iteritems():
    if v >= 0.96:
        toremove.append(k)

In [ ]:
len(toremove)

In [ ]:
toremove

In [ ]:
train.drop('OCCUP_ALL_NEW', axis=1, inplace=True)
test.drop('OCCUP_ALL_NEW', axis=1, inplace=True)

In [ ]:
train.drop(toremove, inplace=True, axis=1)
test.drop(toremove, inplace=True, axis=1)

In [ ]:
for c in train.select_dtypes(include=['object']).columns:
    if c not in ['CUSTOMER_ID','RESPONDERS']:
        # print(c)
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

In [ ]:
train.head()

In [ ]:
train.fillna(-999, inplace=True)
test.fillna(-999, inplace=True)

### normalise the values

In [ ]:
train.fillna(-1, inplace=True)
test.fillna(-1, inplace=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
mxsc = MinMaxScaler()

In [ ]:
for c in train.columns:
    if c not in ['RESPONDERS','CUSTOMER_ID']:
        train[c] = mxsc.fit_transform(train[c])
        test[c] = mxsc.fit_transform(test[c])

In [ ]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold

In [ ]:
train['RESPONDERS'] = train['RESPONDERS'].replace(to_replace = {'Y':1, 'N':0})
target = train['RESPONDERS']

In [ ]:
train.head()

In [ ]:
### add pca features

### Do PCA and get new features

In [ ]:
feature_names = [x for x in train.columns if x not in ['CUSTOMER_ID','RESPONDERS']]

In [ ]:
dim_train = train[feature_names]
dim_test = test[feature_names]

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
rb = StandardScaler()

In [ ]:
dim_train = rb.fit_transform(dim_train)
dim_test = rb.fit_transform(dim_test)

In [ ]:
n_comp = 10

In [ ]:
### data has already been transformed
dim_train = train[feature_names].copy()
dim_test = test[feature_names].copy()

In [ ]:
tsvd = TruncatedSVD(n_components=n_comp, random_state=42)
tsvd_train = tsvd.fit_transform(dim_train)
tsvd_test = tsvd.transform(dim_test)

In [ ]:
pca = PCA(n_components=n_comp, random_state=420)
pca_train = pca.fit_transform(dim_train)
pca_test = pca.transform(dim_test)

In [ ]:
ica = FastICA(n_components=n_comp, random_state=2030)
ica_train = ica.fit_transform(dim_train)
ica_test = ica.transform(dim_test)

In [ ]:
grp = GaussianRandomProjection(n_components=n_comp, random_state=42)
grp_train = grp.fit_transform(dim_train)
grp_test = grp.transform(dim_test)

In [ ]:
srp = SparseRandomProjection(n_components=n_comp, random_state=42)
srp_train = srp.fit_transform(dim_train)
srp_test = srp.transform(dim_test)

In [ ]:
tsne = TSNE(n_components=n_comp, random_state=42)
tsne_train = tsne.fit_transform(dim_train)
tsne_test = test.transform(dim_test)

In [ ]:
for i in range(1, n_comp + 1):
    train['pca_' + str(i)] = pca_train[:,i-1]
    test['pca_' + str(i)] = pca_test[:,i-1]
    
    train['tsvd_' + str(i)] = tsvd_train[:,i-1]
    test['tsvd_' + str(i)] = tsvd_test[:,i-1]
    
    train['ica_' + str(i)] = ica_train[:,i-1]
    test['ica_' + str(i)] = ica_test[:,i-1]
    
    train['grp_' + str(i)] = grp_train[:,i-1]
    test['grp_' + str(i)] = grp_test[:,i-1]
    
    train['srp_' + str(i)] = srp_train[:,i-1]
    test['srp_' + str(i)] = srp_test[:,i-1]
    
    train['tsne_' + str(i)] = tsne_train[:,i-1]
    test['tsne_' + str(i)] = tsne_test[:,i-1]

In [ ]:
del dim_train, dim_test

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.to_pickle('submissions/train_standard.pkl')
test.to_pickle('submissions/test_standard.pkl')

##  Single XGB Model

### XGB 1

In [ ]:
feature_names = [x for x in train.columns if x not in ['CUSTOMER_ID','RESPONDERS']]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size = 0.3, stratify = target)

In [ ]:
params = {"objective": "binary:logistic",
          "booster": "gbtree",
          "nthread": 4,
          "eta": 0.1, # 0.1
          "max_depth": 5, # 7
          "subsample": 0.8,
          "colsample_bytree": 0.4,
          "min_child_weight": 2**3,
          "seed": 2016, 
          "tree_method": "exact",
         "eval_metric":"auc"}

In [ ]:
#dtrain = xgb.DMatrix(X_train[feature_names], y_train, missing=-999)
#dvalid = xgb.DMatrix(X_valid[feature_names], y_valid, missing=-999)
#dtest = xgb.DMatrix(test[feature_names], missing=-999)

## normalised data
dtrain = xgb.DMatrix(X_train[feature_names], y_train)
dvalid = xgb.DMatrix(X_valid[feature_names], y_valid)
dtest = xgb.DMatrix(test[feature_names])

In [ ]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [ ]:
clf1 = xgb.train(params, dtrain, num_boost_round=1000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
xgb.plot_importance(clf1)

In [ ]:
## important features sorted
imp_feats = sorted(clf1.get_fscore(), key=lambda x: x[1], reverse=True)

In [ ]:
preds1 = clf1.predict(dtest)

In [ ]:

sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = preds1

In [ ]:
sub.to_csv('submissions/xgb_single_four.csv', index=False) ## 0.642 / 89.09

In [ ]:
sub.to_csv('submissions/xgb_single_five.csv', index=False) ## 66 / 89.19

In [ ]:
sub.to_csv('submissions/xgb_single_six.csv', index=False) ### 0.65.66 LB / 89.54

In [ ]:
## trying rank average
sub['ranks'] = np.argsort(sub['RESPONDERS'])+1

In [ ]:
maxrank = np.max(sub['ranks'])

In [ ]:
sub['ranks'] = sub['ranks'] / maxrank

In [ ]:
sub.drop('RESPONDERS', axis=1, inplace=True)

In [ ]:
sub.rename(columns={'ranks':'RESPONDERS'}, inplace=True)

In [ ]:
sub.to_csv('submissions/rank_average_xgb_four.csv', index=False) # 0.10273

### XGB with imp. features from previous model - Not good

In [ ]:
len(imp_feats)

In [ ]:
top_100 = imp_feats[:100]

In [ ]:
dtrain = xgb.DMatrix(X_train[top_100], y_train, missing=-999)
dvalid = xgb.DMatrix(X_valid[top_100], y_valid, missing=-999)
dtest = xgb.DMatrix(test[top_100], missing=-999)

In [ ]:
params = {"objective": "binary:logistic",
          "booster": "gbtree",
          "nthread": 4,
          "eta": 0.1, # 0.1
          "max_depth": 5, # 7
          "subsample": 0.8,
          "colsample_bytree": 0.8,
          "min_child_weight": 2,
          "seed": 2016, 
          "tree_method": "exact",
         "eval_metric":"auc"}

In [ ]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [ ]:
clf1 = xgb.train(params, dtrain, num_boost_round=10000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

By removing features, AUC is dropping.

### XGB with Logistic Regression

In [ ]:
## normalised data
dtrain = xgb.DMatrix(X_train[feature_names], y_train)
dvalid = xgb.DMatrix(X_valid[feature_names], y_valid)
dtest = xgb.DMatrix(test[feature_names])

In [ ]:
params = {"objective": "binary:logistic",
          "booster": "gblinear",
          "nthread": 4,
          "alpha": 2**3,
          "lambda": 2**4,
         "eval_metric":"auc"}

In [ ]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [ ]:
clf1 = xgb.train(params, dtrain, num_boost_round=10000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

In [ ]:
preds1 = clf1.predict(dtest)
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = preds1

In [ ]:
sub.to_csv('submissions/xgb_logistic_eight.csv', index=False) # 0.59038 / # 0.8475

## submit average of xgb single five and logistic

In [ ]:
xgb_five = pd.read_csv('submissions/xgb_single_five.csv')

In [ ]:
xgb_five['New_Responders'] = sub['RESPONDERS']

In [ ]:
xgb_five.head()

In [ ]:
xgb_five['average'] = xgb_five.loc[:,['RESPONDERS','New_Responders']].mean(axis=1)

In [ ]:
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = xgb_five['average']

In [ ]:
sub.to_csv('submissions/xgb_best_logistic_average.csv', index=False) ## 0.6606 best score.

### XGB SINGLE MODEL WITH WEIGHTS

### XGB 2

In [ ]:
train['weights'] = train['RESPONDERS'].map(lambda x: 0.8 if x == 1 else 0)

In [ ]:
feature_names2 = [x for x in train.columns if x not in ['CUSTOMER_ID','RESPONDERS']]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size = 0.3, stratify = target)

In [ ]:
params = {"objective": "binary:logistic",
          "booster": "gbtree",
          "nthread": 4,
          "eta": 0.1, # 0.1
          "max_depth": 5, # 7
          "subsample": 0.8,
          "colsample_bytree": 0.4,
          "min_child_weight": 2**3,
          "seed": 2016, 
          "tree_method": "exact",
         "eval_metric":"auc"}

In [ ]:
dtrain = xgb.DMatrix(X_train[feature_names2], y_train, weight=weights)
dvalid = xgb.DMatrix(X_valid[feature_names2], y_valid, weight=weights)
dtest = xgb.DMatrix(test[feature_names])

In [ ]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [ ]:
clf1 = xgb.train(params, dtrain, num_boost_round=10000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

default params are giving lower score than tuned

### EXTRA TREES

In [ ]:
train.drop('weights', axis=1, inplace=True)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer

In [ ]:
et = ExtraTreesClassifier(n_estimators=100, 
                          criterion = 'entropy', 
                          max_depth=10, 
                          max_features='auto',
                          min_samples_split = 3,
                          n_jobs = -1,
                         random_state = 4343)
                         #class_weight = {0:1, 1:10})

In [ ]:
feature_names2 = [x for x in feature_names2 if 'weights' not in x]

In [ ]:
cross_val_score(et, train[feature_names2], target, scoring=make_scorer(roc_auc_score), cv = 5)

### XGB Random Forest

In [ ]:
param_dart = {'booster': 'dart',
         'max_depth': 5, 'learning_rate': 0.1,
         'objective': 'binary:logistic',
            'sample_type': 'uniform',
         'normalize_type': 'tree',
         'rate_drop': 0.1,
         'skip_drop': 0.5,
             'eval_metric':'auc'}

In [ ]:
param_forest = {'booster': 'dart',
         'max_depth': 5, 'learning_rate': 0.1,
         'objective': 'binary:logistic',
            'sample_type': 'uniform',
         'normalize_type': 'forest',
         'rate_drop': 0.2,
         'skip_drop': 0.6,
             'eval_metric':'auc'}

In [ ]:
#dtrain = xgb.DMatrix(X_train[feature_names], y_train, missing=-999)
#dvalid = xgb.DMatrix(X_valid[feature_names], y_valid, missing=-999)
#dtest = xgb.DMatrix(test[feature_names], missing=-999)

## normalised data
dtrain = xgb.DMatrix(X_train[feature_names], y_train)
dvalid = xgb.DMatrix(X_valid[feature_names], y_valid)
dtest = xgb.DMatrix(test[feature_names])

In [ ]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [ ]:
clf1 = xgb.train(param_forest, dtrain, num_boost_round=1000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

In [ ]:
preds1 = clf1.predict(dtest)
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = preds1

In [ ]:
sub.to_csv('submissions/xgb_randomforest.csv', index=False) ## CV - 89.3244 | LB 0.651 ## type == 'tree'

In [ ]:
sub.to_csv('submissions/xgb_randomforest_forest.csv', index=False) ## CV - 89.3427 | LB 0.64629 ## type == 'forest'

## Stacker - Random Forest - with scaled data

In [ ]:
nfolds = 10
skf = StratifiedKFold(n_splits=nfolds, shuffle=True, random_state=9087)

In [ ]:
param_dart = {'booster': 'dart',
         'max_depth': 5, 'learning_rate': 0.1,
         'objective': 'binary:logistic',
            'sample_type': 'uniform',
         'normalize_type': 'tree',
         'rate_drop': 0.1,
         'skip_drop': 0.5,
             'eval_metric':'auc'}

In [ ]:
oof_train = pd.DataFrame({'CUSTOMER_ID':train['CUSTOMER_ID'], 'RESPONDERS':0})
allpredictions = pd.DataFrame()
score = []

In [ ]:
## without upscaling

increase = True
for i, (train_index, test_index) in enumerate(skf.split(train, target)):
    print('Fold %d/%d'%(i+1, nfolds))
    X_train, X_valid = train.iloc[train_index], train.iloc[test_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[test_index]
    
#     if increase:
#         pos = pd.Series(target == 1)

#         X_train = pd.concat([X_train, X_train[pos]], axis=0)
#         y_train = pd.concat([y_train, y_train.loc[pos]], axis=0)
        
#         idx = np.arange(len(X_train))
#         np.random.shuffle(idx)
#         X_train = X_train.iloc[idx]
#         y_train = y_train.iloc[idx]
    
    dtrain = xgb.DMatrix(X_train[feature_names], label=y_train)
    dvalid = xgb.DMatrix(X_valid[feature_names], label=y_valid)
    
    watchlist = [(dtrain, 'train'),(dvalid,'valid')]
    
    clf = xgb.train(param_dart,
                    dtrain,
                    num_boost_round=10000,
                    evals=watchlist,
                    early_stopping_rounds=40,
                    verbose_eval=20,
                    maximize = True)
    
    pred1 = clf.predict(dvalid)
    oof_train.loc[test_index, 'RESPONDERS'] = pred1
    
    scr = roc_auc_score(y_valid, pred1)
    dtest = xgb.DMatrix(test[feature_names])
    preds2 = clf.predict(dtest)
    
    allpredictions['p'+str(i)] = preds2
    score.append(scr)
    
    del X_train, X_valid, y_train, y_valid, clf, pred1, preds2, scr

In [ ]:
np.mean(score)

In [ ]:
oof_train.to_csv('submissions/oof_preds/xgb_train1.csv', index=False)

In [ ]:
allpredictions.head()

In [ ]:
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = allpredictions.mean(axis=1).values

In [ ]:
sub.to_csv('submissions/xgb_bagging_seven.csv', index=False) # 0.58

In [ ]:
train.shape

In [ ]:
test.shape

### XGB 2 - remove zero variance features

In [ ]:
## remove zero variance columns
var_check = pd.DataFrame(columns=['name','variance'])

for i, c in enumerate(train.columns):
    if c not in ['CUSTOMER_ID','RESPONDERS']:
        var_check.loc[i, 'name'] = c
        var_check.loc[i, 'variance'] = train[c].var()
        
var_check.sort_values('variance', ascending=True).reset_index(drop=True)

In [ ]:
cols_to_use = var_check[var_check['variance'] > 1]['name'].tolist() ## remove columns with variance lesser than zero

In [ ]:
params = {"objective": "binary:logistic",
          "booster": "gbtree",
          "nthread": 4,
          "eta": 0.1,
          "max_depth": 5,
          "subsample": 0.8,
          "colsample_bytree": 0.3,
          "min_child_weight": 0,
          "seed": 2017, 
          "tree_method": "exact",
         "eval_metric":"auc"}

In [ ]:
dtrain = xgb.DMatrix(X_train[cols_to_use], y_train, missing=-1)
dvalid = xgb.DMatrix(X_valid[cols_to_use], y_valid, missing=-1)
dtest = xgb.DMatrix(test[cols_to_use], missing=-1)

In [ ]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [ ]:
clf1 = xgb.train(params, dtrain, num_boost_round=10000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

In [ ]:
preds1 = clf1.predict(dtest)

In [ ]:
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = preds1

In [ ]:
sub.to_csv('submissions/xgb_single_three.csv', index=False) ## 54 - score dropping.

### XGB BAGGED

In [ ]:
oof_train = pd.DataFrame({'CUSTOMER_ID':train['CUSTOMER_ID'], 'RESPONDERS':0})
allpredictions = pd.DataFrame()
score = []

In [ ]:
increase = True
for i, (train_index, test_index) in enumerate(skf.split(train, target)):
    print('Fold %d/%d'%(i+1, nfolds))
    X_train, X_valid = train.iloc[train_index], train.iloc[test_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[test_index]
    
    if increase:
        pos = pd.Series(target == 1)

        X_train = pd.concat([X_train, X_train[pos]], axis=0)
        y_train = pd.concat([y_train, y_train.loc[pos]], axis=0)
        
        idx = np.arange(len(X_train))
        np.random.shuffle(idx)
        X_train = X_train.iloc[idx]
        y_train = y_train.iloc[idx]
    
    dtrain = xgb.DMatrix(X_train[feature_names], label=y_train, missing=-1)
    dvalid = xgb.DMatrix(X_valid[feature_names], label=y_valid, missing = -1)
    
    watchlist = [(dtrain, 'train'),(dvalid,'valid')]
    
    clf = xgb.train(params,
                    dtrain,
                    num_boost_round=10000,
                    evals=watchlist,
                    early_stopping_rounds=40,
                    verbose_eval=20,
                    maximize = True)
    
    pred1 = clf.predict(dvalid)
    oof_train.loc[test_index, 'RESPONDERS'] = pred1
    
    scr = roc_auc_score(y_valid, pred1)
    dtest = xgb.DMatrix(test[feature_names], missing= - 1)
    preds2 = clf.predict(dtest)
    
    allpredictions['p'+str(i)] = preds2
    score.append(scr)
    
    del X_train, X_valid, y_train, y_valid, clf, pred1

In [ ]:
np.mean(score)

In [ ]:
oof_train.to_csv('submissions/oof_preds/xgb_train1.csv', index=False)

In [ ]:
allpredictions.head()

In [ ]:
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = allpredictions.mean(axis=1).values

In [ ]:
sub.to_csv('submissions/xgb_bagging_seven.csv', index=False) # 0.58